In [4]:
import uuid

import pprint
import logging
import boto3

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [13]:
input_text:str = "I would like to book a flight"
agent_id:str = 'ZXO44HCY0H'                                             # note this from the agent console on Bedrock
agent_alias_id:str = 'TSTALIASID'                                       # fixed for draft version of the agent
session_id:str = str(uuid.uuid1())                                      # random identifier
enable_trace:bool = True

In [14]:
session = boto3.Session(profile_name='bach-dev', region_name='us-east-1')
client = session.client(service_name="bedrock-agent-runtime")

logger.info(client)

[2024-03-31 15:10:29,730] p2212 {276264083.py:4} INFO - <botocore.client.AgentsforBedrockRuntime object at 0x00000172E46C0A90>


In [15]:
# invoke the agent API
response = client.invoke_agent(
    inputText=input_text,
    agentId=agent_id,
    agentAliasId=agent_alias_id,
    sessionId=session_id,
    enableTrace=enable_trace
)

logger.info(pprint.pprint(response))

[2024-03-31 15:10:32,005] p2212 {899143767.py:10} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Sun, 31 Mar 2024 21:10:30 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '18db9813-efa3-11ee-aafc-c8348e0cfaf7',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'f2999671-3e89-4e9e-9a0d-92a6dfcebb2f'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'f2999671-3e89-4e9e-9a0d-92a6dfcebb2f',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x00000172E455B070>,
 'contentType': 'application/json',
 'sessionId': '18db9813-efa3-11ee-aafc-c8348e0cfaf7'}


In [16]:
%%time
import json
event_stream = response['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}") 
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2024-03-31 15:10:33,736] p2212 {<timed exec>:12} INFO - {
  "agentAliasId": "TSTALIASID",
  "agentId": "ZXO44HCY0H",
  "sessionId": "18db9813-efa3-11ee-aafc-c8348e0cfaf7",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationInput": {
        "inferenceConfiguration": {
          "maximumLength": 2048,
          "stopSequences": [
            "\n\nHuman:"
          ],
          "temperature": 0.0,
          "topK": 250,
          "topP": 1.0
        },
        "text": "You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<tools>\n    <tool_description>\n<tool_name>GET::TravelActionGroup::getAirportCode</tool

[2024-03-31 15:10:36,435] p2212 {<timed exec>:12} INFO - {
  "agentAliasId": "TSTALIASID",
  "agentId": "ZXO44HCY0H",
  "sessionId": "18db9813-efa3-11ee-aafc-c8348e0cfaf7",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationOutput": {
        "parsedResponse": {
          "isValid": true,
          "rationale": "The input \"I would like to book a flight\" seems to be a reasonable request to book a flight. It does not appear malicious or trying to manipulate my behavior. Since I have been provided the tools to assist with booking flights, this input falls into Category D for questions I can assist with using my available tools."
        },
        "traceId": "f2999671-3e89-4e9e-9a0d-92a6dfcebb2f-pre-0"
      }
    }
  }
}
[2024-03-31 15:10:36,450] p2212 {<timed exec>:12} INFO - {
  "agentAliasId": "TSTALIASID",
  "agentId": "ZXO44HCY0H",
  "sessionId": "18db9813-efa3-11ee-aafc-c8348e0cfaf7",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationInput": {
        "in

CPU times: total: 15.6 ms
Wall time: 10.6 s


In [14]:
# Response from the agent
print(agent_answer)

Here are the available flight options from Calgary to Paris on April 1st:

Flight 123: 
Departing YEG at 11:00pm on March 31, layover in YYC, departing YYC at 3:19am on April 1, arriving CDG at 11:12am on April 1.
Total price: $617.43

Flight 346:
Departing YEG at 12:15pm on March 31, layover in YYC, departing YYC at 3:19am on April 1, arriving CDG at 11:12am on April 1.  
Total price: $567.43

Would you like to book one of these flights? I can use your saved credit card and reward dollars if you want to book.
